<a href="https://colab.research.google.com/github/Champioa00/LAB1/blob/master/LAB2V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
## LAB 2 ##
## Anthony Champion ##
## IST 718 ##

In [23]:
## Load required packages ##
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
#from botocore.client import Config
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from fbprophet import Prophet
import statsmodels.api as sm
from  multiprocessing import Pool, cpu_count
import pandas as pd
import numpy as np
#import ibm_boto3
import itertools
import types
import math
import time

sns.set(rc={'figure.figsize':(11.7,8.27)})
%matplotlib inline 

DATA Cleaning:

Let's start with importing the difffernt data sets.  From there we can explore and perform the needed data cleansing. 



In [24]:
df_data_1 = pd.read_csv('Zip_zhvi_uc_sfr_month.csv')
df_data_1.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01,1996-02,1996-03,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,1997-10,1997-11,1997-12,1998-01,1998-02,1998-03,1998-04,1998-05,1998-06,1998-07,...,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.792330e+06,2.687480e+06,2.750319e+06,2.745687e+06,2.672237e+06,2.703466e+06,2.799758e+06,2.762261e+06,2.972422e+06,2.964929e+06,2.941399e+06,2.770281e+06,2.709474e+06,2.807152e+06,2.757996e+06,2.898636e+06,2.883489e+06,2.813502e+06,2.835291e+06,2.864990e+06,2.710262e+06,2.851837e+06,2.915106e+06,2.778842e+06,2.812505e+06,2.801768e+06,2.872429e+06,2869819.021,2.792227e+06,2.880981e+06,2.936416e+06,3.033201e+06,2.904437e+06,2.801494e+06,2.721834e+06,2.640603e+06,2.616576e+06,2.600348e+06,2.563603e+06,2.729760e+06
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,327184.4715,327713.4837,325984.4579,332778.2610,334414.5751,337601.7483,337856.7732,340445.1863,346222.3082,348798.2996,353450.8352,352990.2028,354387.5651,357231.4176,359681.4492,361963.9914,372552.6507,369575.8297,373462.4970,348818.2141,341442.7292,323433.5647,315459.2506,316542.5223,295871.2783,300335.2839,300666.0235,309593.5198,315575.7585,329702.7681,340326.4415,...,7.892213e+05,7.910212e+05,7.899114e+05,7.805394e+05,7.872967e+05,7.919914e+05,7.900941e+05,7.925490e+05,7.910878e+05,7.864774e+05,7.838324e+05,7.801943e+05,7.750737e+05,7.740944e+05,7.725794e+05,7.801559e+05,7.915183e+05,7.910302e+05,7.915584e+05,7.939067e+05,7.876053e+05,7.908220e+05,7.917972e+05,7.999406e+05,8.017769e+05,7.938478e+05,7.989722e+05,797140.864,8.138223e+05,8.136838e+05,8.204414e+05,8.171122e+05,8.237994e+05,8.301982e+05,8.286601e+05,8.252674e+05,8.381829e+05,8.389180e+05,8.381373e+05,8.495258e+05
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.306724e+06,5.303907e+06,5.307117e+06,5.156562e+06,5.249348e+06,5.182788e+06,5.058869e+06,5.166357e+06,5.043388e+06,5.041397e+06,4.900863e+06,4.846561e+06,4.806427e+06,4.780078e+06,4.760010e+06,4.874589e+06,4.783694e+06,4.670615e+06,4.821009e+06,4.831602e+06,4.917240e+06,4.823658e+06,4.681429e+06,4.742684e+06,4.750099e+06,4.737670e+06,4.734036e+06,4503091.435,4.820726e+06,5.032885e+06,5.072752e+06,5.104681e+06,4.991198e+06,5.204816e+06,5.430261e+06,5.489555e+06,5.183702e+06,5.143927e+06,5.115112e+06,5.111999e+06
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,178597.7329,182239.3716,181934.9870,179668.4547,184266.4870,183117.2403,188706.1648,183779.9983,186036.3819,182704.3914,184768.1298,187562.7857,189633.2751,189402.0634,193389.9616,199159.9068,199053.6127,201124.2151,205113.1101,209095.7221,211203.2530,207095.6308,207235.3069,204503.8534,203883.6108,205334.1647,203134.1089,203749.4133,203032.1247,203485.1372,202113.1230,...,3.530590e+05,3.541628e+05,3.525020e+05,3.554251e+05,3.556854e+05,3.553540e+05,3.555060e+05,3.568726e+05,3.563078e+05,3.568684e+05,3.551285e+05,3.563217e+05,3.550922e+05,3.540023e+05,3.537106e+05,3.547125e+05,3.549502e+05,3.582089e+05,3.608014e+05,3.609959e+05,3.610848e+05,3.610934e+05,3.634250e+05,3.641847e+05,3.671116e+05,3.720254e+05,3.768957e+05,380199.342,3.884185e+05,3.970397e+05,4.039702e+05,4.138433e+05,4.270610e+05,4.386916e+05,4.466750e+05,4.535979e+05,4.605603e+05

In [72]:
## Check to see if the data was loaded correctly ##
na_values = df_data_1.isna().sum()
print(na_values)

RegionID      0
SizeRank      0
RegionName    0
RegionType    0
StateName     0
             ..
2021-09       1
2021-10       1
2021-11       1
2021-12       1
2022-01       1
Length: 322, dtype: int64


In [73]:
## Load in metro data from the US Department of Labor ##
metro_area = pd.read_csv('OD.csv')
metro_area.head()


,ZIP CODE,STATE,MSA No.,GPCI_W,GPCI_PE,GPCI_ME,County No.,MSA Name
0,210,NH,14460.0,1.0,1.046,0.658,15.0,"Boston-Cambridge-Quincy, MA-NH MSA"
1,211,NH,14460.0,1.0,1.046,0.658,15.0,"Boston-Cambridge-Quincy, MA-NH MSA"
2,212,NH,14460.0,1.0,1.046,0.658,15.0,"Boston-Cambridge-Quincy, MA-NH MSA"
3,213,NH,14460.0,1.0,1.046,0.658,15.0,"Boston-Cambridge-Quincy, MA-NH MSA"
4,214,NH,14460.0,1.0,1.046,0.658,15.0,"Boston-Cambridge-Quincy, MA-NH MSA"


In [74]:
# Keep data with missing metro name. 
missing_metro = df_data_1[df_data_1['Metro'].isna()]
missing_index = missing_metro.index

# Drop observation without Metro data. These observations will then be reattached
df_metro_complete = df_data_1.drop(index=missing_index)

# Keep zip code and msa name from the metro_area data set
metro_area_simp = metro_area[['ZIP CODE', 'MSA Name']]

# Merge missing metro with metro_area_simp 
metro_complete = missing_metro.merge(metro_area_simp, how='left', left_on='RegionName', right_on='ZIP CODE')

In [78]:
# Check to see that there are no NaN variables in the new column. 
print(metro_complete['MSA Name'].isna().sum())

# There are no NaN values now. We'll now replace the Metro column with the MSA Name column, drop the overlapping columns, and 
# reattach the observations to the df_data_1 data set. 
metro_complete['Metro'] = metro_complete['MSA Name']
metro_complete1 = metro_complete.iloc[:, 0:322]

# Reattach data to original data. 
df_data_1 = df_metro_complete.append(metro_complete1)
print(df_data_1.shape)

0
(7246, 322)


In [57]:
# Obtain column names of year-month combinations. 
year_month = df_data_1.iloc[:, 9:].columns.values

df_data_2 = df_data_1

# Iterate over year-month combination, replacing NaN with average value for metropolitan area 
for ym in df_data_2[year_month]:
    df_data_2[ym] = df_data_2.groupby('Metro')[ym].transform(lambda x: x.fillna(x.mean()))

In [58]:
df_data_2.describe()

,RegionID,SizeRank,RegionName,1996-01,1996-02,1996-03,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,1997-10,1997-11,1997-12,1998-01,1998-02,1998-03,1998-04,1998-05,1998-06,1998-07,1998-08,1998-09,1998-10,1998-11,1998-12,1999-01,...,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01
count,7246.000000,7246.000000,7246.000000,6.326000e+03,6.335000e+03,6.335000e+03,6.341000e+03,6.361000e+03,6.362000e+03,6.366000e+03,6.382000e+03,6.387000e+03,6.387000e+03,6.392000e+03,6.394000e+03,6.421000e+03,6.421000e+03,6.425000e+03,6.450000e+03,6.454000e+03,6.454000e+03,6.456000e+03,6.456000e+03,6.456000e+03,6.463000e+03,6.483000e+03,6.483000e+03,6.483000e+03,6.483000e+03,6.483000e+03,6.539000e+03,6.547000e+03,6.566000e+03,6.566000e+03,6.566000e+03,6.566000e+03,6.567000e+03,6.567000e+03,6.567000e+03,6.575000e+03,...,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.245000e+03,7.246000e+03,7.246000e+03,7.246000e+03,7.246000e+03,7.246000e+03,7.246000e+03,7.246000e+03
mean,83555.852056,3644.122136,51936.146012,1.477499e+05,1.484798e+05,1.483922e+05,1.487604e+05,1.498241e+05,1.501866e+05,1.505555e+05,1.512078e+05,1.515044e+05,1.519172e+05,1.522357e+05,1.526185e+05,1.524674e+05,1.526700e+05,1.532282e+05,1.536188e+05,1.545901e+05,1.552662e+05,1.560757e+05,1.565483e+05,1.570174e+05,1.571475e+05,1.577243e+05,1.583979e+05,1.592410e+05,1.597726e+05,1.604788e+05,1.609444e+05,1.621635e+05,1.631732e+05,1.642035e+05,1.652999e+05,1.666761e+05,1.674678e+05,1.682728e+05,1.689461e+05,1.687638e+05,...,3.773547e+05,3.779239e+05,3.783407e+05,3.785693e+05,3.796896e+05,3.812216e+05,3.827298e+05,3.844501e+05,3.849533e+05,3.827546e+05,3.828088e+05,3.831893e+05,3.834522e+05,3.840986e+05,3.860580e+05,3.886000e+05,3.911651e+05,3.943967e+05,3.962670e+05,3.945128e+05,3.962581e+05,3.989478e+05,4.025751e+05,4.065623e+05,4.108476e+05,4.141361e+05,4.181919e+05,4.223499e+05,4.284716e+05,4.345747e+05,4.427894e+05,4.535946e+05,4.628812e+05,4.701375e+05,4.745721e+05,4.786331e+05,4.831447e+05,4.881975e+05,4.949445e+05,5.029324e+05
std,36996.315051,2108.956358,29501.924802,9.603880e+04,1.170349e+05,1.149727e+05,1.172795e+05,1.268436e+05,1.293124e+05,1.311689e+05,1.345639e+05,1.355468e+05,1.377713e+05,1.401812e+05,1.407235e+05,1.394554e+05,1.364362e+05,1.365676e+05,1.355342e+05,1.371216e+05,1.406213e+05,1.455398e+05,1.489726e+05,1.458905e+05,1.480395e+05,1.508771e+05,1.507618e+05,1.540065e+05,1.551925e+05,1.567974e+05,1.620988e+05,1.680138e+05,1.724209e+05,1.787761e+05,1.750145e+05,1.800670e+05,1.808128e+05,1.837522e+05,1.874878e+05,1.809796e+05,...,4.670569e+05,4.650521e+05,4.631138e+05,4.586396e+05,4.579245e+05,4.596185e+05,4.580674e+05,4.635852e+05,4.649186e+05,4.564693e+05,4.536881e+05,4.513870e+05,4.493126e+05,4.480017e+05,4.515612e+05,4.580348e+05,4.588557e+05,4.591798e+05,4.627660e+05,4.565417e+05,4.583446e+05,4.560117e+05,4.568523e+05,4.593226e+05,4.583453e+05,4.568445e+05,4.606017e+05,4.578205e+05,4.643001e+05,4.680449e+05,4.785890e+05,4.897202e+05,4.952003e+05,5.052962e+05,5.070870e+05,5.111801e+05,5.129065e+05,5.160924e+05,5.218747e+05,5.296139e+05
min,58004.000000,0.000000,602.000000,9.590703e+03,9.479815e+03,9.556451e+03,9.628464e+03,9.370827e+03,9.089543e+03,8.598643e+03,8.423374e+03,8.534906e+03,8.144440e+03,7.532039e+03,7.952085e+03,7.919122e+03,7.

In [59]:
df_data_2.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01,1996-02,1996-03,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,1997-10,1997-11,1997-12,1998-01,1998-02,1998-03,1998-04,1998-05,1998-06,1998-07,...,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,210903.874358,229360.661072,228020.216261,229394.713047,239510.766705,240836.912482,242283.795664,244467.352018,245046.711011,246676.515626,248672.614477,249187.266023,249656.477897,249120.10962,250035.021225,250217.483717,251339.821036,254028.352201,257592.203428,259945.486045,258699.216769,260772.525075,265198.433632,266468.054188,273550.464974,273365.033819,274738.184937,278334.772754,282440.470416,284925.012388,288340.245935,...,2.792330e+06,2.687480e+06,2.750319e+06,2.745687e+06,2.672237e+06,2.703466e+06,2.799758e+06,2.762261e+06,2.972422e+06,2.964929e+06,2.941399e+06,2.770281e+06,2.709474e+06,2.807152e+06,2.757996e+06,2.898636e+06,2.883489e+06,2.813502e+06,2.835291e+06,2.864990e+06,2.710262e+06,2.851837e+06,2.915106e+06,2.778842e+06,2.812505e+06,2.801768e+06,2.872429e+06,2869819.021,2.792227e+06,2.880981e+06,2.936416e+06,3.033201e+06,2.904437e+06,2.801494e+06,2.721834e+06,2.640603e+06,2.616576e+06,2.600348e+06,2.563603e+06,2.729760e+06
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,327184.471500,327713.483700,325984.457900,332778.261000,334414.575100,337601.748300,337856.773200,340445.186300,346222.308200,348798.299600,353450.835200,352990.202800,354387.565100,357231.41760,359681.449200,361963.991400,372552.650700,369575.829700,373462.497000,348818.214100,341442.729200,323433.564700,315459.250600,316542.522300,295871.278300,300335.283900,300666.023500,309593.519800,315575.758500,329702.768100,340326.441500,...,7.892213e+05,7.910212e+05,7.899114e+05,7.805394e+05,7.872967e+05,7.919914e+05,7.900941e+05,7.925490e+05,7.910878e+05,7.864774e+05,7.838324e+05,7.801943e+05,7.750737e+05,7.740944e+05,7.725794e+05,7.801559e+05,7.915183e+05,7.910302e+05,7.915584e+05,7.939067e+05,7.876053e+05,7.908220e+05,7.917972e+05,7.999406e+05,8.017769e+05,7.938478e+05,7.989722e+05,797140.864,8.138223e+05,8.136838e+05,8.204414e+05,8.171122e+05,8.237994e+05,8.301982e+05,8.286601e+05,8.252674e+05,8.381829e+05,8.389180e+05,8.381373e+05,8.495258e+05
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,210903.874358,229360.661072,228020.216261,229394.713047,239510.766705,240836.912482,242283.795664,244467.352018,245046.711011,246676.515626,248672.614477,249187.266023,249656.477897,249120.10962,250035.021225,250217.483717,251339.821036,254028.352201,257592.203428,259945.486045,258699.216769,260772.525075,265198.433632,266468.054188,273550.464974,273365.033819,274738.184937,278334.772754,282440.470416,284925.012388,288340.245935,...,5.306724e+06,5.303907e+06,5.307117e+06,5.156562e+06,5.249348e+06,5.182788e+06,5.058869e+06,5.166357e+06,5.043388e+06,5.041397e+06,4.900863e+06,4.846561e+06,4.806427e+06,4.780078e+06,4.760010e+06,4.874589e+06,4.783694e+06,4.670615e+06,4.821009e+06,4.831602e+06,4.917240e+06,4.823658e+06,4.681429e+06,4.742684e+06,4.750099e+06,4.737670e+06,4.734036e+06,4503091.435,4.820726e+06,5.032885e+06,5.072752e+06,5.104681e+06,4.991198e+06,5.204816e+06,5.430261e+06,5.489555e+06,5.183702e+06,5.143927e+06,5.115112e+06,5.111999e+06
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,178597.732900,182239.371600,181934.987000,179668.454700,184266.487000,183117.240300,188706.164800,183779.998300,186036.381900,182704.391400,184768.129800,187562.785700,189633.

In [124]:
# Columns that want to be kept
to_keep = ['RegionName']

# Columns to be converted to observations
to_obs = df_data_2.iloc[:,9:]

# Pivot the data to have on observation for each region for each year-month. 
df_data_pivot = pd.pivot_table(df_data_2, columns=to_keep, values=to_obs)

# Reset the index and rename year-month and values columns for easy identification.
df_data_pivot = df_data_pivot.reset_index()
df_data_pivot.rename(columns={'level_0':'date', 1:'value'}, inplace=True)

# Convert data column to datetime
df_data_pivot['index'] = pd.to_datetime(df_data_pivot['index'], infer_datetime_format=True)


# Print first five rows
df_data_pivot.sort_values(['index']).head()


RegionName,index,602,612,662,693,717,725,745,791,917,924,926,949,953,956,957,985,1001,1002,1013,1020,1027,1040,1056,1060,1075,1085,1089,1104,1108,1109,1201,1247,1301,1420,1440,1453,1501,1545,1550,...,98942,98944,99004,99016,99037,99163,99201,99202,99203,99205,99206,99207,99208,99212,99216,99217,99223,99224,99301,99336,99337,99352,99354,99362,99403,99501,99502,99504,99507,99508,99515,99516,99517,99577,99645,99654,99701,99705,99709,99801
0,1996-01-01,133836.195920,NaN,133836.195920,99311.927966,118871.16320,NaN,87861.913710,NaN,NaN,99311.927966,NaN,NaN,NaN,NaN,111640.650522,NaN,118460.7970,176567.1996,92222.24066,99265.50514,125310.4606,97882.03284,76705.14266,173482.8798,133205.1592,118595.7594,116442.1398,72195.88025,81958.14658,69102.11507,100739.4438,85884.43218,97151.52707,96652.21313,93410.46956,120685.8848,122173.7232,122276.3707,100961.5901,...,NaN,NaN,138229.3319,135297.9245,163501.5428,NaN,100005.23540,86190.21102,147122.7637,91239.10484,130583.0190,79469.30804,152633.3836,111279.4364,115363.2879,121482.0128,167978.4131,181324.6826,99344.21478,99344.21478,99344.21478,99344.21478,99344.21478,103754.5034,NaN,111497.5702,136391.5939,115553.1551,128085.2878,113521.0047,160218.4172,202814.2105,136047.6792,153787.7347,139768.517033,139768.517033,NaN,NaN,NaN,NaN
1,1996-02-01,133582.933764,NaN,133582.933764,98757.503739,120137.09140,NaN,87677.628965,NaN,NaN,98757.503739,NaN,NaN,NaN,NaN,112130.318164,NaN,118631.9315,176586.9652,92324.74278,100160.44150,124936.5505,97890.13069,76651.27340,172471.6292,131948.5165,118947.3470,115792.3150,72389.02606,81523.80084,68551.28097,100654.5498,85925.12917,96738.26443,97609.23712,94659.49182,121481.2344,121877.1472,123003.4604,101728.6833,...,NaN,NaN,139771.6618,135969.5236,163946.0110,NaN,99931.95901,85668.13469,147368.5818,90889.38082,130693.8609,79428.11401,152428.7049,111585.0366,115337.6139,121683.2560,168190.9492,180377.0520,99660.78622,99660.78622,99660.78622,99660.78622,99660.78622,103714.6144,NaN,112380.7952,136921.0956,115842.2668,130240.2083,113699.3863,164020.0674,210636.9163,136091.8321,154926.3110,141639.875444,141639.875444,NaN,NaN,NaN,NaN
2,1996-03-01,133638.841798,NaN,133638.841798,98520.811167,120724.53440,NaN,88518.377760,NaN,NaN,98520.811167,NaN,NaN,NaN,NaN,112231.859983,NaN,120127.3008,177257.3589,93682.74072,101155.08710,125176.5801,98403.15063,76756.12879,174918.6078,131783.8134,119415.0912,115502.2480,71886.22332,82472.25253,69788.44746,100524.8282,85973.75569,96850.60515,97164.13835,93726.74033,121423.8724,121677.2799,122239.9318,101163.5030,...,NaN,NaN,141123.8466,136522.8509,163500.8042,NaN,99333.14876,85039.44107,145923.5672,89817.00991,130261.3757,78154.81424,151124.2300,111723.1164,114891.6427,121322.6430,167401.6169,180918.1522,99999.35050,99999.35050,99999.35050,99999.35050,99999.35050,103725.6282,NaN,114390.7594,136756.5611,119752.0433,129487.3143,115748.0570,161676.3031,207390.0191,135767.4738,160928.7327,142433.029311,142433.029311,NaN,NaN,NaN,NaN
3,1996-04-01,133335.193510,NaN,133335.193510,98377.927586,121754.14935,NaN,87543.770430,NaN,NaN,98377.927586,NaN,NaN,NaN,NaN,112436.060105,NaN,120009.2743,177295.6388,92978.10063,101628.88660,125541.7073,98544.57867,77212.47977,173250.6682,131488.1385,120383.5624,115732.0176,70949.03741,81344.37368,70071.91512,100623.3876,86674.12730,96867.75429,97829.91394,94732.26096,122646.7874,122024.6318,123822.1482,103268.7938,...,NaN,NaN,140714.2909,137462.6699,164224.8998,NaN,99825.79733,85232.24352,146637.7243,90252.73493,130854.9155,78262.75952,151398.9631,111897.5154,115610.4972,122816.8110,167344.8438,179998.0821,98601.23279,98601.23279,98601.23279,98601.23279,98601.23279,103689.6337,NaN,114921.3662,137750.4671,121293.8055,130632.2148,116223.7617,164488.1288,212206.9443,137292.7544,166160.8014,144552.249356,144552.249356,NaN,NaN,NaN,NaN
4,1996-05-01,133626.838368,NaN,133626.838368,98405.781811,122498.94255,NaN,87676.444465,NaN,NaN,98405.781811,NaN,NaN,NaN,NaN,112876.184708,NaN,118897.6000,177442.8574,93465.32790,10167

In [126]:
print(df_data_pivot.shape)
print(df_data_pivot.dtypes)

(313, 7247)
RegionName
index    datetime64[ns]
602             float64
612             float64
662             float64
693             float64
              ...      
99654           float64
99701           float64
99705           float64
99709           float64
99801           float64
Length: 7247, dtype: object


In [117]:
# Print the first and last date of the data 
print('Oldest observation: ', df_data_pivot['index'].min())
print('Newest observation: ', df_data_pivot['index'].max(), '\n')

Oldest observation:  1996-01-01 00:00:00
Newest observation:  2022-01-01 00:00:00 



In [120]:
df_data_pivot = df_data_pivot.reset_index()

In [119]:
# Create a new data frame to average value per date. Keep observations starting from December 2008 to plot. 
national = df_data_pivot.groupby('index')['value'].mean()
national = national.reset_index()
national.columns = ['index', 'avg_value']
national = national[national['index']>='2022-01-01']
print(national.head())

# Plot the data as a line plot. 
national.plot(x='date', y='avg_value')
_ = plt.xlabel('Date')
_ = plt.ylabel('Avg Value')
_ = plt.title('Avg Value by Date')
plt.show()

KeyError: ignored